# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 1 2022 1:40PM,243169,15,8322277-BO,"Carwin Pharmaceutical Associates, LLC",Released
1,Aug 1 2022 1:24PM,243168,10,CTF0011345,"Citieffe, Inc.",Released
2,Aug 1 2022 1:24PM,243168,10,CTF0011346,"Citieffe, Inc.",Released
3,Aug 1 2022 1:24PM,243168,10,CTF0011347,"Citieffe, Inc.",Released
4,Aug 1 2022 1:01PM,243167,10,SO94337,"Senseonics, Incorporated",Released
5,Aug 1 2022 1:01PM,243167,10,SO94338,"Senseonics, Incorporated",Released
6,Aug 1 2022 12:59PM,243166,10,8347588,Beach Products Inc,Released
7,Aug 1 2022 12:59PM,243166,10,8347589,Beach Products Inc,Released
8,Aug 1 2022 12:58PM,243165,21,622239,"NBTY Global, Inc.",Executing
9,Aug 1 2022 12:57PM,243164,10,Enova-10389,Emerginnova,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,243165,Executing,1
40,243166,Released,2
41,243167,Released,2
42,243168,Released,3
43,243169,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243165,NaN,1.0,NaN
243166,NaN,NaN,2.0
243167,NaN,NaN,2.0
243168,NaN,NaN,3.0
243169,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243085,0.0,0.0,1.0
243091,0.0,1.0,0.0
243101,0.0,0.0,1.0
243110,0.0,0.0,3.0
243123,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243085,0,0,1
243091,0,1,0
243101,0,0,1
243110,0,0,3
243123,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243085,0,0,1
1,243091,0,1,0
2,243101,0,0,1
3,243110,0,0,3
4,243123,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243085,,,1
1,243091,,1,
2,243101,,,1
3,243110,,,3
4,243123,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 1 2022 1:40PM,243169,15,"Carwin Pharmaceutical Associates, LLC"
1,Aug 1 2022 1:24PM,243168,10,"Citieffe, Inc."
4,Aug 1 2022 1:01PM,243167,10,"Senseonics, Incorporated"
6,Aug 1 2022 12:59PM,243166,10,Beach Products Inc
8,Aug 1 2022 12:58PM,243165,21,"NBTY Global, Inc."
9,Aug 1 2022 12:57PM,243164,10,Emerginnova
14,Aug 1 2022 12:53PM,243163,12,Hush Hush
16,Aug 1 2022 12:53PM,243162,12,Hush Hush
21,Aug 1 2022 12:43PM,243161,10,"Snap Medical Industries, LLC"
33,Aug 1 2022 12:42PM,243160,19,"NAPP Technologies, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 1 2022 1:40PM,243169,15,"Carwin Pharmaceutical Associates, LLC",,,1
1,Aug 1 2022 1:24PM,243168,10,"Citieffe, Inc.",,,3
2,Aug 1 2022 1:01PM,243167,10,"Senseonics, Incorporated",,,2
3,Aug 1 2022 12:59PM,243166,10,Beach Products Inc,,,2
4,Aug 1 2022 12:58PM,243165,21,"NBTY Global, Inc.",,1,
5,Aug 1 2022 12:57PM,243164,10,Emerginnova,,,5
6,Aug 1 2022 12:53PM,243163,12,Hush Hush,,2,
7,Aug 1 2022 12:53PM,243162,12,Hush Hush,,4,1
8,Aug 1 2022 12:43PM,243161,10,"Snap Medical Industries, LLC",,,12
9,Aug 1 2022 12:42PM,243160,19,"NAPP Technologies, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 1 2022 1:40PM,243169,15,"Carwin Pharmaceutical Associates, LLC",1,,
1,Aug 1 2022 1:24PM,243168,10,"Citieffe, Inc.",3,,
2,Aug 1 2022 1:01PM,243167,10,"Senseonics, Incorporated",2,,
3,Aug 1 2022 12:59PM,243166,10,Beach Products Inc,2,,
4,Aug 1 2022 12:58PM,243165,21,"NBTY Global, Inc.",,1,
5,Aug 1 2022 12:57PM,243164,10,Emerginnova,5,,
6,Aug 1 2022 12:53PM,243163,12,Hush Hush,,2,
7,Aug 1 2022 12:53PM,243162,12,Hush Hush,1,4,
8,Aug 1 2022 12:43PM,243161,10,"Snap Medical Industries, LLC",12,,
9,Aug 1 2022 12:42PM,243160,19,"NAPP Technologies, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 1 2022 1:40PM,243169,15,"Carwin Pharmaceutical Associates, LLC",1,,
1,Aug 1 2022 1:24PM,243168,10,"Citieffe, Inc.",3,,
2,Aug 1 2022 1:01PM,243167,10,"Senseonics, Incorporated",2,,
3,Aug 1 2022 12:59PM,243166,10,Beach Products Inc,2,,
4,Aug 1 2022 12:58PM,243165,21,"NBTY Global, Inc.",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 1 2022 1:40PM,243169,15,"Carwin Pharmaceutical Associates, LLC",1.0,NaN,NaN
1,Aug 1 2022 1:24PM,243168,10,"Citieffe, Inc.",3.0,NaN,NaN
2,Aug 1 2022 1:01PM,243167,10,"Senseonics, Incorporated",2.0,NaN,NaN
3,Aug 1 2022 12:59PM,243166,10,Beach Products Inc,2.0,NaN,NaN
4,Aug 1 2022 12:58PM,243165,21,"NBTY Global, Inc.",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 1 2022 1:40PM,243169,15,"Carwin Pharmaceutical Associates, LLC",1.0,0.0,0.0
1,Aug 1 2022 1:24PM,243168,10,"Citieffe, Inc.",3.0,0.0,0.0
2,Aug 1 2022 1:01PM,243167,10,"Senseonics, Incorporated",2.0,0.0,0.0
3,Aug 1 2022 12:59PM,243166,10,Beach Products Inc,2.0,0.0,0.0
4,Aug 1 2022 12:58PM,243165,21,"NBTY Global, Inc.",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3404094,222.0,7.0,33.0
12,486325,1.0,6.0,0.0
15,729456,17.0,30.0,0.0
16,729433,10.0,0.0,0.0
19,1702044,15.0,16.0,0.0
20,1215569,49.0,7.0,0.0
21,243165,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3404094,222.0,7.0,33.0
1,12,486325,1.0,6.0,0.0
2,15,729456,17.0,30.0,0.0
3,16,729433,10.0,0.0,0.0
4,19,1702044,15.0,16.0,0.0
5,20,1215569,49.0,7.0,0.0
6,21,243165,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,222.0,7.0,33.0
1,12,1.0,6.0,0.0
2,15,17.0,30.0,0.0
3,16,10.0,0.0,0.0
4,19,15.0,16.0,0.0
5,20,49.0,7.0,0.0
6,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,222.0
1,12,Released,1.0
2,15,Released,17.0
3,16,Released,10.0
4,19,Released,15.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,33.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,7.0,6.0,30.0,0.0,16.0,7.0,1.0
Released,222.0,1.0,17.0,10.0,15.0,49.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,33.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,7.0,6.0,30.0,0.0,16.0,7.0,1.0
2,Released,222.0,1.0,17.0,10.0,15.0,49.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,33.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,7.0,6.0,30.0,0.0,16.0,7.0,1.0
2,Released,222.0,1.0,17.0,10.0,15.0,49.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()